In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
file_name = "pe_ratio.xlsx"

current_dir = os.getcwd()  # Gets the directory where this script/notebook is run
file_path = os.path.join(current_dir, file_name)

# Load the Excel file and sheet
xls = pd.ExcelFile(file_path)
df_pe = pd.read_excel(xls, sheet_name="PE_ratio_hist")  # Adjust if the sheet name changes

# Ensure datetime index is set properly
df_pe['Dates'] = pd.to_datetime(df_pe['Dates'], errors='coerce')
df_pe.set_index('Dates', inplace=True)

# Preview the DataFrame
df_pe.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Gautier Pellerin\\Documents\\Code\\AMP_Algorithmic_Trading\\strategies\\pe_ratio.xlsx'

In [ ]:
#Extracting Apple P/E
pe_aapl = df_pe[['AAPL UW Equity']].dropna()
pe_aapl = pe_aapl.rename(columns={'AAPL UW Equity': 'PE_Ratio'})

In [ ]:
#Weekly data to ensure comparability as in the dummy strategy
pe_aapl_weekly = pe_aapl.resample('W-FRI').last()

In [ ]:
#Compare range as in the dummy strategy
pe_aapl_weekly= pe_aapl_weekly.loc['2014-12-12':'2016-12-12']

In [ ]:
#Calculating Mean Over the reference Range
pe_value_mean = pe_aapl_weekly.mean()

In [ ]:
default_std = pe_aapl_weekly.std()

In [ ]:
def generate_signals(pe_series, window, k, pe_value=pe_value_mean, default_std=default_std):
    # 1. Rolling stats
    rolling_mean = pe_series.rolling(window=window).mean()
    rolling_std = pe_series.rolling(window=window).std()

    # 2. Fill missing values with default assumptions
    rolling_mean_filled = rolling_mean.fillna(pe_value)
    rolling_std_filled = rolling_std.fillna(default_std)

    # 3. Create upper/lower bands
    upper_band = rolling_mean_filled + k * rolling_std_filled
    lower_band = rolling_mean_filled - k * rolling_std_filled

    # 4. Generate trading signals
    signals = pd.Series(0, index=pe_series.index)
    signals[pe_series < lower_band] = 1   # Long
    signals[pe_series > upper_band] = -1  # Short

    # 5. Return filled values (used for plotting) and signals
    return signals, rolling_mean_filled, upper_band, lower_band

In [ ]:
signals, rolling_mean, upper_band, lower_band = generate_signals(
    pe_aapl_weekly["PE_Ratio"], window=5, k=0.5
)

In [ ]:
def get_weekly_prices(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)  # auto_adjust=True is default
    data.index = pd.to_datetime(data.index)
    weekly_prices = data['Close'].resample('W-FRI').last()
    return weekly_prices.dropna()

In [ ]:
TICKER = "AAPL"
START_DATE = "2014-12-12"
END_DATE = "2016-12-12"

In [ ]:
weekly_prices=get_weekly_prices(TICKER, START_DATE,END_DATE)
weekly_prices.head(5)

In [ ]:
returns = weekly_prices.pct_change().fillna(0)

In [ ]:
#Ensuring it will be 1 DImensional
if isinstance(returns, pd.DataFrame):
    returns = returns['AAPL']

In [ ]:
strategy_returns = signals.shift(1) * returns 

In [ ]:
cumulative_returns = (1 + strategy_returns).cumprod()
cumulative_returns

In [ ]:
results_df = pd.DataFrame({
    "Signal": signals,
    "P/E": pe_aapl_weekly["PE_Ratio"],
    "Upper Band": upper_band,
    "Lower band": lower_band,
    "Return": returns,
    "Strategy_Return": strategy_returns,
    "Cumulative_Return": cumulative_returns
})

In [ ]:
def plot_results(cumulative_returns, pe_series, rolling_mean, upper_band, lower_band, signals):
    # Align series
    signals = signals.loc[pe_series.index]

    fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    # Plot cumulative returns
    axs[0].plot(cumulative_returns, label='Strategy Cumulative Return')
    axs[0].set_title("Value Strategy on AAPL (P/E-based)")
    axs[0].set_ylabel("Cumulative Return")
    axs[0].legend()
    axs[0].grid(True)

    # Plot P/E and signals
    axs[1].plot(pe_series, label='P/E', color='black', alpha=0.6)
    axs[1].plot(rolling_mean, label='Rolling Mean', linestyle='--')
    axs[1].plot(upper_band, label='Upper Band', linestyle=':')
    axs[1].plot(lower_band, label='Lower Band', linestyle=':')

    axs[1].scatter(signals[signals == 1].index, pe_series[signals == 1], label='Long', marker='^', color='green')
    axs[1].scatter(signals[signals == -1].index, pe_series[signals == -1], label='Short', marker='v', color='red')

    axs[1].set_title("Simulated P/E with Signal Bands")
    axs[1].set_ylabel("P/E Ratio")
    axs[1].legend()
    axs[1].grid(True)

    plt.xlabel("Date")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_results(cumulative_returns, pe_aapl_weekly, rolling_mean, upper_band, lower_band, signals)

In [ ]:
# Define base transaction cost
base_cost = 0.001  # 0.1% per trade

# Compute rolling volatility need to choose the time frame
rolling_vol = returns.rolling(window=4).std()

# Normalize volatility relative to mean so to avoid extreme values 
scaled_vol = rolling_vol / rolling_vol.mean()

# Compute dynamic cost
dynamic_cost_per_trade = base_cost * scaled_vol

# Detect position changes (you trade only when signal changes)
position = signals.shift(1)  # Position held at each time step
position_change = position.diff().abs().fillna(0)  # 0→1 or 1→-1 etc.

# Compute transaction costs
transaction_costs = dynamic_cost_per_trade * position_change

# Apply cost to strategy returns
strategy_returns_net = strategy_returns - transaction_costs

# Recalculate cumulative returns
cumulative_returns_net = (1 + strategy_returns_net).cumprod()

# Store everything in the results DataFrame
results_df["Rolling_Vol"] = rolling_vol
results_df["Dynamic_Cost"] = transaction_costs
results_df["Strategy_Return_Net"] = strategy_returns_net
results_df["Cumulative_Return_Net"] = cumulative_returns_net


### The cost structure applies transaction costs only when a position change occurs, such as moving from long to short or from a position to cash. These costs are adjusted dynamically according to market volatility. In periods of high volatility, transaction costs are higher due to wider bid-ask spreads and increased slippage, while in stable periods, costs are lower. This approach creates a more realistic backtest by reflecting the true cost of trading under different market conditions.

In [ ]:
results_df.tail(5)

In [ ]:
# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

# Top plot
axs[0].plot(results_df["Cumulative_Return"], label="Without Transaction Costs", linewidth=2)
axs[0].plot(results_df["Cumulative_Return_Net"], label="With Transaction Costs", linewidth=2, linestyle='--')
axs[0].set_title("Strategy Cumulative Returns: With vs Without Transaction Costs")
axs[0].set_ylabel("Cumulative Return")
axs[0].legend()
axs[0].grid(True)

# Bottom plot
axs[1].plot(results_df["Dynamic_Cost"], label="Volatility-Adjusted Cost per Trade", color='mediumseagreen', linewidth=2)
axs[1].fill_between(results_df.index, results_df["Dynamic_Cost"], alpha=0.3, color='mediumseagreen')
axs[1].set_title("Volatility-Adjusted Transaction Cost Over Time")
axs[1].set_ylabel("Transaction Cost")
axs[1].set_xlabel("Date")
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()